In [2]:
import requests
import json
from bs4 import BeautifulSoup as bs
import pandas as pd

Check concat file

In [3]:
shots = pd.read_csv('/home/axel/Code/Python/axel/streamlit/concat_files/concat_shots.csv')

In [3]:
shots.columns

Index(['id', 'eventType', 'teamId', 'playerId', 'playerName', 'x', 'y', 'min',
       'minAdded', 'isBlocked', 'isOnTarget', 'blockedX', 'blockedY',
       'goalCrossedY', 'goalCrossedZ', 'expectedGoals',
       'expectedGoalsOnTarget', 'shotType', 'situation', 'period', 'isOwnGoal',
       'onGoalShot', 'isSavedOffLine', 'isFromInsideBox', 'keeperId',
       'firstName', 'lastName', 'fullName', 'teamColor', 'matchRound',
       'teamName', 'h_a', 'source_file'],
      dtype='object')

Team with most shots at home

In [13]:
# Filter rows where h_a is 'h'
home_teams = shots[shots['h_a'] == 'h']

# Group by teamName and count occurrences
team_counts = home_teams['teamName'].value_counts()

# Get the team with the highest count
most_frequent_home_team = team_counts.idxmax()
count = team_counts.max()
print(f"The team with most 'h' appearances is {most_frequent_home_team} with {count} occurrences")

The team with most 'h' appearances is Cerro Porteño with 61 occurrences


Filter keepers

In [49]:
shots['keeperId'].nunique()

41

Filter goals

In [50]:
goals = shots[shots['eventType'] == 'Goal']
len(goals)

209

In [22]:
ownGoal = goals[goals['isOwnGoal'] == True]
len(ownGoal)

3

Filter by on target shots

In [63]:
isOnTarget = shots[shots['isOnTarget'] == True]

In [64]:
len(isOnTarget)

1196

Count teams

In [4]:
shots['teamId'].nunique()

32

Count unique players

In [65]:
shots['playerId'].nunique()

488

Count DF

In [66]:
len(shots)

2073

Avg xG

In [32]:
avg_expected_goals = shots['expectedGoals'].mean()
print(f"Average expectedGoals: {avg_expected_goals}")

Average expectedGoals: 0.09737349817459909


In [33]:
print(f"Average expectedGoals (2 decimals): {avg_expected_goals:.2f}")  # 0.10
print(f"Average expectedGoals (3 decimals): {avg_expected_goals:.3f}")  # 0.097

Average expectedGoals (2 decimals): 0.10
Average expectedGoals (3 decimals): 0.097


Check null values on columns | xG

In [62]:
def check_null_columns(df):
    null_columns = df.columns[df.isnull().any()]
    if null_columns.empty:
        print("No columns with null values.")
    else:
        print("Columns with null values:")
        for col in null_columns:
            print(f"{col}: {df[col].isnull().sum()} null values of {df.shape[0]} rows")

# Example usage
check_null_columns(shots)

Columns with null values:
minAdded: 1804 null values of 2073 rows
blockedX: 957 null values of 2073 rows
blockedY: 957 null values of 2073 rows
keeperId: 1340 null values of 2073 rows
firstName: 107 null values of 2073 rows


In [55]:
# check specific column NA
shots[shots['expectedGoals'].isnull()]

,id,eventType,teamId,playerId,playerName,x,y,min,minAdded,isBlocked,...,isFromInsideBox,keeperId,firstName,lastName,fullName,teamColor,matchRound,teamName,h_a,source_file
1688,2815157049,Goal,192875,1349235,Luis Zarate,3.793103,35.29625,26,NaN,False,...,True,923383.0,Luis,Zarate,Luis Zarate,#201818,5,Independiente del Valle,a,river-plate-vs-independiente-del-valle.csv


In [56]:
# fill NA values with 0
shots['expectedGoals'] = shots['expectedGoals'].fillna(0.00)

In [57]:
# fill NA values with 0
shots['expectedGoalsOnTarget'] = shots['expectedGoalsOnTarget'].fillna(0.00)

In [11]:
# check specific column with NA
shots[shots['h_a'].isnull()]

,id,eventType,teamId,playerId,playerName,x,y,min,minAdded,isBlocked,...,isFromInsideBox,keeperId,firstName,lastName,fullName,teamColor,matchRound,teamName,h_a,source_file
374,2798470541,Miss,6453,729089,Byron Castillo,77.191748,20.706242,62,NaN,False,...,False,NaN,Byron,Castillo,Byron Castillo,#DA0203,2,Barcelona de Guayaquil,NaN,river-plate-vs-barcelona-sc.csv
376,2798471685,Miss,6453,207135,Dixon Arroyo,94.385965,37.660000,67,NaN,False,...,True,NaN,Dixon,Arroyo,Dixon Arroyo,#DA0203,2,Barcelona de Guayaquil,NaN,river-plate-vs-barcelona-sc.csv


In [16]:
# find specific row
shots.loc[shots['id'] == 2798470541]

,id,eventType,teamId,playerId,playerName,x,y,min,minAdded,isBlocked,...,isFromInsideBox,keeperId,firstName,lastName,fullName,teamColor,matchRound,teamName,h_a,source_file
374,2798470541,Miss,6453,729089,Byron Castillo,77.191748,20.706242,62,NaN,False,...,False,NaN,Byron,Castillo,Byron Castillo,#DA0203,2,Barcelona de Guayaquil,a,river-plate-vs-barcelona-sc.csv


In [59]:
# Save new DF to CSV after cleaning
shots.to_csv('/home/axel/Code/Python/axel/streamlit/concat_files/concat_shots.csv', index=False)

Most shots taken

In [60]:
shots['teamName'].value_counts().sort_values(ascending=False)

# teams_df = shots['teamName'].value_counts().reset_index()
# teams_df.columns = ['teamName', 'count']
# teams_df.sort_values(by='count', ascending=False)

teamName
Olimpia                                  89
Cerro Porteño                            84
Racing Club                              82
River Plate                              78
Talleres de Córdoba                      78
Palmeiras                                76
Bolívar                                  76
Vélez Sarsfield                          76
Deportivo Táchira                        73
Alianza Lima                             71
Peñarol                                  69
Central Córdoba (Santiago del Estero)    68
Atlético Nacional                        68
Bahia                                    68
Botafogo                                 65
Estudiantes de La Plata                  64
São Paulo                                64
Universidad de Chile                     64
Fortaleza EC                             64
Nacional                                 62
Flamengo                                 61
Atlético Bucaramanga                     60
Sporting Cristal       

Filter by shots on target

In [5]:
target_shots = shots[shots['isOnTarget'] == True]
target_shots.shape

(1196, 33)

In [6]:
# target shots sort values from top to bottom
target_shots['teamName'].value_counts().sort_values(ascending=False)

teamName
Vélez Sarsfield                          54
Olimpia                                  54
Racing Club                              54
River Plate                              51
Palmeiras                                49
Alianza Lima                             44
Bolívar                                  43
Talleres de Córdoba                      43
Peñarol                                  42
Estudiantes de La Plata                  42
Central Córdoba (Santiago del Estero)    42
Cerro Porteño                            41
Botafogo                                 40
Atlético Nacional                        39
Nacional                                 39
Fortaleza EC                             38
Sporting Cristal                         38
Bahia                                    38
Universidad de Chile                     37
Flamengo                                 35
Internacional                            35
Deportivo Táchira                        33
São Paulo              

In [8]:
# target shots sort values from top to bottom with index
target_shots.groupby('teamName')['isOnTarget'].count().reset_index(name='count').sort_values(by='count', ascending=False).reset_index(drop=True)

,teamName,count
0,Olimpia,42
1,Vélez Sarsfield,36
2,Bolívar,34
3,Racing Club,33
4,Atlético Nacional,32
5,Alianza Lima,31
6,Palmeiras,29
7,Estudiantes de La Plata,29
8,Talleres de Córdoba,29
9,Cerro Porteño,27


Group by team & player id options

In [9]:
# Group by 'teamName' and 'playerId' and count occurrences
target_shots.groupby(['teamName', 'playerId']).size().reset_index(name='count')

# Now `grouped_counts_df` holds the result as a DataFrame
# You can print it to see the results:
# print(grouped_counts_df)

,teamName,playerId,count
0,Alianza Lima,18694,1
1,Alianza Lima,33101,5
2,Alianza Lima,127268,3
3,Alianza Lima,166121,1
4,Alianza Lima,200257,2
...,...,...,...
324,Vélez Sarsfield,954660,1
325,Vélez Sarsfield,1083898,1
326,Vélez Sarsfield,1393542,1
327,Vélez Sarsfield,1607035,2


In [10]:
# Group by teamName and playerId, then get the first playerName for each group
target_shots.groupby(['teamName', 'playerId'])['playerName'].first().reset_index()

,teamName,playerId,playerName
0,Alianza Lima,18694,Hernan Barcos
1,Alianza Lima,33101,José Paolo Guerrero
2,Alianza Lima,127268,Pablo Damian Lavandeira
3,Alianza Lima,166121,Pablo Cepellini
4,Alianza Lima,200257,Fernando Gaibor
...,...,...,...
324,Vélez Sarsfield,954660,Matias Pellegrini
325,Vélez Sarsfield,1083898,Augustin Lagos
326,Vélez Sarsfield,1393542,Christian Ordonez
327,Vélez Sarsfield,1607035,Alvaro Montoro


In [9]:
# Group by teamName and playerId, then aggregate:
# - Get the first playerName for the group
# - Get the size (count) of the group
target_shots.groupby(['teamName', 'playerId']).agg(
    playerName=('playerName', 'first'), # Get the first player name found in the group
    count=('playerId', 'size') # Count the number of rows in the group (using 'size') We apply 'size' to an arbitrary column like 'playerId'
).reset_index() # Turns the grouped output (with MultiIndex) back into a DataFrame

,teamName,playerId,playerName,count
0,Alianza Lima,33101,José Paolo Guerrero,1
1,Alianza Lima,127268,Pablo Damian Lavandeira,3
2,Alianza Lima,166121,Pablo Cepellini,1
3,Alianza Lima,200257,Fernando Gaibor,1
4,Alianza Lima,248704,Miguel Trauco,1
...,...,...,...,...
256,Vélez Sarsfield,675102,Michael Santos,3
257,Vélez Sarsfield,723682,Agustin Bouzat,1
258,Vélez Sarsfield,1393542,Christian Ordonez,1
259,Vélez Sarsfield,1607035,Alvaro Montoro,1


DS | Group by team & player id options

In [ ]:
# op 1 Group by team first, then show players
(target_shots.groupby(['teamName', 'playerId', 'playerName'])
    .size()
    .reset_index(name='count')
    .sort_values(['teamName', 'count'], ascending=[True, False]))

,teamName,playerId,playerName,count
6,Alianza Lima,831684,Kevin Quevedo,4
1,Alianza Lima,127268,Pablo Damian Lavandeira,3
7,Alianza Lima,1086550,Ricardo Lagos,2
8,Alianza Lima,1098460,Alan Cantero,2
10,Alianza Lima,1312034,Guillermo Enrique,2
...,...,...,...,...
254,Vélez Sarsfield,523417,Emanuel Mammana,2
252,Vélez Sarsfield,288759,Francisco Pizzini,1
257,Vélez Sarsfield,723682,Agustin Bouzat,1
258,Vélez Sarsfield,1393542,Christian Ordonez,1


In [ ]:
# op 2 Create a hierarchical index (multi-index)
result = (target_shots.groupby(['teamName', 'playerId'])
           .agg(playerName=('playerName', 'first'),
                count=('playerId', 'size')))
# Keep it as a multi-index DataFrame (don't reset_index)
result

playerName  count
teamName        playerId                                
Alianza Lima    33101         José Paolo Guerrero      1
                127268    Pablo Damian Lavandeira      3
                166121            Pablo Cepellini      1
                200257            Fernando Gaibor      1
                248704              Miguel Trauco      1
...                                           ...    ...
Vélez Sarsfield 675102             Michael Santos      3
                723682             Agustin Bouzat      1
                1393542         Christian Ordonez      1
                1607035            Alvaro Montoro      1
                1618860             Maher Carrizo      4

[261 rows x 2 columns]

In [8]:
# op 3 Format the output for better readability
result = (target_shots.groupby(['teamName', 'playerId'])
           .agg(playerName=('playerName', 'first'),
                count=('playerId', 'size'))
           .reset_index()
           .sort_values(['teamName', 'count'], ascending=[True, False]))

# Print with team headers
current_team = None
for _, row in result.iterrows():
    if row['teamName'] != current_team:
        print(f"\n=== {row['teamName']} ===")
        current_team = row['teamName']
    print(f"{row['playerName']}: {row['count']} shots")

result


=== Alianza Lima ===
Kevin Quevedo: 8 shots
José Paolo Guerrero: 5 shots
Pablo Damian Lavandeira: 3 shots
Fernando Gaibor: 2 shots
Eryc Castillo: 2 shots
Ricardo Lagos: 2 shots
Alan Cantero: 2 shots
Guillermo Enrique: 2 shots
Hernan Barcos: 1 shots
Pablo Cepellini: 1 shots
Miguel Trauco: 1 shots
Erick Noriega: 1 shots
Marco Huaman: 1 shots

=== Atlético Bucaramanga ===
Luciano Pons: 6 shots
Frank Castaneda: 5 shots
Carlos Henao: 4 shots
Fabian Sambueza: 3 shots
Kevin Londono: 2 shots
Sherman Cardenas: 1 shots
Leonardo Flores: 1 shots
Aldair Zarate: 1 shots

=== Atlético Nacional ===
Alfredo Morelos: 8 shots
Marino Hinestroza: 7 shots
Edwin Cardona: 3 shots
Dairon Asprilla: 3 shots
Felipe Roman: 2 shots
Kevin Viveros: 2 shots
Juan Zapata: 2 shots
William Tesillo: 1 shots
Jorman Campuzano: 1 shots
Juan Jose Arias: 1 shots
Kevin Parra: 1 shots
Elkin Rivero: 1 shots

=== Bahia ===
Luciano Rodriguez: 6 shots
Jean Lucas: 5 shots
Erick Pulga: 3 shots
Santiago Ramos: 2 shots
Willian José: 1 s

,teamName,playerId,playerName,count
7,Alianza Lima,831684,Kevin Quevedo,8
1,Alianza Lima,33101,José Paolo Guerrero,5
2,Alianza Lima,127268,Pablo Damian Lavandeira,3
4,Alianza Lima,200257,Fernando Gaibor,2
6,Alianza Lima,458861,Eryc Castillo,2
...,...,...,...,...
327,Vélez Sarsfield,1607035,Alvaro Montoro,2
317,Vélez Sarsfield,288759,Francisco Pizzini,1
324,Vélez Sarsfield,954660,Matias Pellegrini,1
325,Vélez Sarsfield,1083898,Augustin Lagos,1


In [ ]:
# op 4 Create separate DataFrames per team
team_groups = target_shots.groupby('teamName')
for team, group in team_groups:
    print(f"\n=== {team} ===")
    player_counts = group['playerName'].value_counts().reset_index()
    player_counts.columns = ['playerName', 'count']
    print(player_counts.to_string(index=False))


=== Alianza Lima ===
             playerName  count
          Kevin Quevedo      4
Pablo Damian Lavandeira      3
          Ricardo Lagos      2
      Guillermo Enrique      2
           Alan Cantero      2
          Eryc Castillo      1
           Marco Huaman      1
        Pablo Cepellini      1
          Miguel Trauco      1
    José Paolo Guerrero      1
        Fernando Gaibor      1

=== Atlético Bucaramanga ===
     playerName  count
Frank Castaneda      4
Fabian Sambueza      3
   Luciano Pons      2
  Kevin Londono      2
  Aldair Zarate      1
   Carlos Henao      1

=== Atlético Nacional ===
       playerName  count
Marino Hinestroza      7
  Alfredo Morelos      6
      Juan Zapata      2
    Edwin Cardona      2
    Kevin Viveros      2
  Dairon Asprilla      2
  Juan Jose Arias      1
     Felipe Roman      1
  William Tesillo      1

=== Bahia ===
          playerName  count
          Jean Lucas      5
   Luciano Rodriguez      3
      Santiago Ramos      2
         Er

Top 10 players with most shots on target

In [11]:
target_shots.groupby(['playerName', 'teamName']).size().reset_index(name='Shots').sort_values('Shots', ascending=False).head(10)

,playerName,teamName,Shots
31,Braian Romero,Vélez Sarsfield,10
291,Ramiro Vaca,Bolívar,9
19,Alfredo Morelos,Atlético Nacional,8
163,Javier Correa,Colo-Colo,8
3,Adrian Martinez,Racing Club,8
209,Kevin Quevedo,Alianza Lima,8
249,Marino Hinestroza,Atlético Nacional,7
95,Fabio Gomes,Bolívar,7
314,Sebastian Driussi,River Plate,7
242,Maher Carrizo,Vélez Sarsfield,7
